In [6]:
#ETL Asal Kota

import pandas as pd
import mysql.connector

# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('dataetl.csv')

# Memilih hanya kolom 'Asal Daerah'
df = df[['asal_daerah']]

# Menghapus baris duplikat berdasarkan 'Asal Daerah'
df_no_duplicates = df.drop_duplicates()

# Menambahkan kolom 'ID'
df_no_duplicates['id_daerah'] = range(1, len(df_no_duplicates) + 1)

# Menampilkan hasil
print(df_no_duplicates)

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in df_no_duplicates.iterrows():
            cursor.execute("INSERT INTO dim_asal_daerah (id_daerah, asal_daerah) VALUES (%s, %s)", (row['id_daerah'], row['asal_daerah']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")


            asal_daerah  id_daerah
0            Kota Solok          1
1       Kabupaten Solok          2
29  Kabupaten Sijunjung          3
80          Daerah Lain          4
97      Kota Sawahlunto          5
Data berhasil dimuat ke MySQL
Koneksi ke MySQL ditutup


C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\495879974.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['id_daerah'] = range(1, len(df_no_duplicates) + 1)


In [9]:
import pandas as pd
import mysql.connector

# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('dataetl.csv')

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        # Mendapatkan data wilayah dari database
        query = "SELECT asal_daerah, id_daerah FROM dim_asal_daerah"  # Ganti nama_tabel_wilayah
        df_wilayah = pd.read_sql(query, connection)
        
        # Memilih kolom 'MR', 'Sex', 'Umur', dan 'Asal Daerah'
        df_selected = df[['mr', 'sex', 'umur', 'asal_daerah']]

        # Join dengan tabel wilayah untuk mendapatkan 'ID Daerah'
        df_selected = pd.merge(df_selected, df_wilayah, how='left', on='asal_daerah')

        # Menambahkan kolom 'Range Umur'
        bins = [0, 4, 9, 18, 60, float('inf')]
        labels = ['0-4', '5-9', '10-18', '19-59', '60+']
        df_selected['range_umur'] = pd.cut(df_selected['umur'], bins=bins, labels=labels, right=False)

        # Menyisipkan kolom 'Range Umur' di sebelah kanan kolom 'Umur'
        df_selected.insert(df_selected.columns.get_loc('umur') + 1, 'range_umur', df_selected.pop('range_umur'))

        # Menghapus kolom 'asal_daerah'
        df_selected.drop(columns=['asal_daerah'], inplace=True)

        # Menghapus baris yang merupakan duplikat berdasarkan kolom 'mr'
        df_selected.drop_duplicates(subset=['mr'], inplace=True)
        
        # Menampilkan hasil
        print(df_selected)

        # Membuat kursor untuk eksekusi perintah SQL
        cursor = connection.cursor()

        # Melakukan iterasi baris DataFrame dan memasukkan data ke dalam tabel MySQL
        for index, row in df_selected.iterrows():
            cursor.execute("INSERT INTO dim_pasien (mr, sex, umur, id_daerah, range_umur) VALUES (%s, %s, %s, %s, %s)", 
                           (row['mr'], row['sex'], row['umur'], row['id_daerah'], row['range_umur']))
            connection.commit()

        print("Data berhasil dimuat ke MySQL ke tabel dim_pasien")
          
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")


C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\3186586264.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_wilayah = pd.read_sql(query, connection)


           mr sex  umur range_umur  id_daerah
0      193427   L    33      19-59          1
1      165437   P    60        60+          2
2      199411   P    67        60+          2
3      160862   P    41      19-59          2
4      199518   P    53      19-59          2
...       ...  ..   ...        ...        ...
31189  183559   P    54      19-59          2
31190  249065   P    66        60+          2
31193  146967   P    51      19-59          2
31194  249636   P    46      19-59          3
31195  236659   P    69        60+          1

[24895 rows x 5 columns]
Data berhasil dimuat ke MySQL ke tabel dim_pasien
Koneksi ke MySQL ditutup


In [11]:
#ETL Cara Bayar

import pandas as pd
import mysql.connector

# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('dataetl.csv')

# Memilih hanya kolom 'Cara Bayar'
df = df[['cara_bayar']]

# Menghapus baris duplikat berdasarkan 'Cara Bayar'
df_no_duplicates = df.drop_duplicates()

# Menambahkan kolom 'ID'
df_no_duplicates['id_cb'] = range(1, len(df_no_duplicates) + 1)

# Menampilkan hasil
print(df_no_duplicates)


try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in df_no_duplicates.iterrows():
            cursor.execute("INSERT INTO dim_cara_bayar (id_cb, cara_bayar) VALUES (%s, %s)", (row['id_cb'], row['cara_bayar']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")

             cara_bayar  id_cb
0                  Umum      1
2          BPJS Non PBI      2
3              BPJS PBI      3
175       PT Bukit Asam      4
812     Dana Pendamping      5
1511             BAZNAZ      6
2986                PLN      7
8271       Jasa Raharja      8
8646           InHealth      9
9206          Sijunjung     10
12016  Sawahlunto Sehat     11
15298         Jampersal     12
15309          COVID 19     13
16340     PT Kereta Api     14
22015           BPJS TK     15
24032            Telkom     16
Data berhasil dimuat ke MySQL
Koneksi ke MySQL ditutup


C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\695969993.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['id_cb'] = range(1, len(df_no_duplicates) + 1)


In [13]:
#ETL Bangsal

import pandas as pd
import mysql.connector

# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('dataetl.csv')

# Memilih hanya kolom 'Bangsal'
df = df[['bangsal']]

# Menghapus baris duplikat berdasarkan 'Bangsal'
df_no_duplicates = df.drop_duplicates()

# Menambahkan kolom 'ID'
df_no_duplicates['id_bangsal'] = range(1, len(df_no_duplicates) + 1)

# Menampilkan hasil
print(df_no_duplicates)


try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in df_no_duplicates.iterrows():
            cursor.execute("INSERT INTO dim_bangsal (id_bangsal, bangsal) VALUES (%s, %s)", (row['id_bangsal'], row['bangsal']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")

                bangsal  id_bangsal
0               Interne           1
6459         Bedah Umum           2
10495   Bedah Orthopedi           3
12047         Neurologi           4
14602         Kebidanan           5
16259              Mata           6
16327               THT           7
16943               ICU           8
18515              CVCU           9
19357              Anak          10
21867      Perinatologi          11
22884            Pusako          12
25392              Paru          13
26610           Jantung          14
28884         Psikiatri          15
30169  Gigi Bedah Mulut          16
30433    Bedah Digestif          17
30813     Kulit Kelamin          18
30820               HCU          19
Data berhasil dimuat ke MySQL
Koneksi ke MySQL ditutup


C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\1047961793.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_duplicates['id_bangsal'] = range(1, len(df_no_duplicates) + 1)


In [27]:
#ETL Cara Keluar

import pandas as pd
import mysql.connector

# Membaca file CSV ke dalam DataFrame
df = pd.read_csv('dataetl.csv')

# Memilih hanya kolom 'Cara Keluar'
df['cara_keluar']

# Menghapus baris duplikat berdasarkan 'Cara Keluar'
df_no_duplicates = df[['cara_keluar']].drop_duplicates()

# Mengurutkan data berdasarkan kolom 'cara_keluar' secara alfabetis
df_no_duplicates = df_no_duplicates.sort_values(by='cara_keluar', ascending=True)

# Menambahkan kolom 'ID'
df_no_duplicates['id_ck'] = range(1, len(df_no_duplicates) + 1)

# Menampilkan hasil
print(df_no_duplicates)

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in df_no_duplicates.iterrows():
            cursor.execute("INSERT INTO dim_cara_keluar (id_ck, cara_keluar) VALUES (%s, %s)", (row['id_ck'], row['cara_keluar']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")

          cara_keluar  id_ck
6447     Boleh Pulang      1
19348      M < 48 JAM      2
6307       M > 48 JAM      3
99          Meninggal      4
26                PAP      5
0      Pindah Ruangan      6
34              Rujuk      7
1               Sehat      8
Data berhasil dimuat ke MySQL
Koneksi ke MySQL ditutup


In [25]:
#ETL Diagnosa

import pandas as pd
import mysql.connector

# Membaca file CSV
df = pd.read_csv('dataetl.csv')

# Mengambil data dari kolom 'diagnosa1', 'diagnosa2', dan 'diagnosa3'
diagnosa_df = df[['diagnosa1', 'diagnosa2', 'diagnosa3']]

# Menggabungkan data menjadi satu kolom 'diagnosa'
diagnosa_series = diagnosa_df.stack().reset_index(drop=True)

# Menghapus duplikat
diagnosa_series = diagnosa_series.drop_duplicates()

# Menambahkan kolom 'id_diagnosa'
diagnosa_df_final = pd.DataFrame({'id_diagnosa': range(1, len(diagnosa_series) + 1),
                                  'diagnosa': diagnosa_series})

# Menampilkan hasil
print(diagnosa_df_final)

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in diagnosa_df_final.iterrows():
            cursor.execute("INSERT INTO dim_diagnosa (id_diagnosa, diagnosa) VALUES (%s, %s)", (row['id_diagnosa'], row['diagnosa']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")

       id_diagnosa               diagnosa
0                1               Susp hiv
1                2                Dispnea
2                3           Anemia berat
3                4                   Dhf 
4                5                 Tipoid
...            ...                    ...
41471         1751                  Psmba
41700         1752          Hipokoagulasi
41723         1753            Pansitopeni
41759         1754                 Uremic
41900         1755  Efek toksik pestisida

[1755 rows x 2 columns]
Data berhasil dimuat ke MySQL
Koneksi ke MySQL ditutup


In [78]:
import pandas as pd
import mysql.connector

# Membaca data dari file CSV untuk tabel fakta
data_etl = pd.read_csv("dataetl.csv", usecols=['mr', 'cara_keluar', 'cara_bayar', 'bangsal', 'diagnosa1', 'diagnosa2', 'diagnosa3', 'tanggal'])

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
       # Membaca data dari MySQL untuk setiap dimensi
        dim_bangsal = pd.read_sql("SELECT * FROM dim_bangsal", connection)
        dim_cara_bayar = pd.read_sql("SELECT * FROM dim_cara_bayar", connection)
        dim_cara_keluar = pd.read_sql("SELECT * FROM dim_cara_keluar", connection)
        dim_diagnosa = pd.read_sql("SELECT * FROM dim_diagnosa", connection)

        # Lakukan join antara tabel fakta dan dimensi dari MySQL
        data_etl = pd.merge(data_etl, dim_bangsal, how="left", on="bangsal").drop(columns=['bangsal'])
        data_etl = pd.merge(data_etl, dim_cara_bayar, how="left", on="cara_bayar").drop(columns=['cara_bayar'])
        data_etl = pd.merge(data_etl, dim_cara_keluar, how="left", on="cara_keluar").drop(columns=['cara_keluar'])
        data_etl = pd.merge(data_etl, dim_diagnosa, how="left", left_on="diagnosa1", right_on="diagnosa").drop(columns=['diagnosa1', 'diagnosa']).rename(columns={'id_diagnosa': 'id_diag1'})
        data_etl = pd.merge(data_etl, dim_diagnosa, how="left", left_on="diagnosa2", right_on="diagnosa").drop(columns=['diagnosa2', 'diagnosa']).rename(columns={'id_diagnosa': 'id_diag2'})
        data_etl = pd.merge(data_etl, dim_diagnosa, how="left", left_on="diagnosa3", right_on="diagnosa").drop(columns=['diagnosa3', 'diagnosa']).rename(columns={'id_diagnosa': 'id_diag3'})

        # Tambahkan kolom ID rawatan menggunakan indeks baris
        data_etl['id_rawatan'] = data_etl.index + 1

        # Ubah format kolom tanggal
        data_etl['tanggal'] = pd.to_datetime(data_etl['tanggal']).dt.strftime('%Y-%m-%d')

        print(data_etl)

        # Membuat kursor untuk eksekusi perintah SQL
        cursor = connection.cursor()

        # Melakukan iterasi baris DataFrame dan memasukkan data ke dalam tabel MySQL
        for index, row in data_etl.iterrows():
            cursor.execute("INSERT INTO fakta_rawatan (id_rawatan,tanggal,mr,id_bangsal,id_cb,id_ck,id_diag1,id_diag2,id_diag3) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", 
                           (row['id_rawatan'], row['tanggal'], row['mr'], row['id_bangsal'], row['id_cb'], row['id_ck'], row['id_diag1'], row['id_diag2'], row['id_diag3']))
            connection.commit()

        print("Data berhasil dimuat ke MySQL ke tabel fakta_rawatan")

        
          
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")


          tanggal      mr  id_bangsal  id_cb  id_ck  id_diag1  id_diag2  \
0      2020-01-01  193427           1      1      6       1.0       NaN   
1      2020-01-01  165437           1      1      8       2.0       NaN   
2      2020-01-01  199411           1      2      8       3.0       NaN   
3      2020-01-01  160862           1      3      8       4.0       NaN   
4      2020-01-01  199518           1      2      8       5.0       NaN   
...           ...     ...         ...    ...    ...       ...       ...   
31193  2023-09-28  146967          19      3      8       6.0     174.0   
31194  2023-09-29  249636          19      2      8      10.0     328.0   
31195  2023-09-30  236659          19      3      3     117.0      76.0   
31196  2023-09-30  243093          19      3      8      58.0       6.0   
31197  2023-09-30   85171          19      2      8     101.0      23.0   

       id_diag3  id_rawatan  
0           NaN           1  
1           NaN           2  
2        

C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\696343404.py:16: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_bangsal = pd.read_sql("SELECT * FROM dim_bangsal", connection)
C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\696343404.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_cara_bayar = pd.read_sql("SELECT * FROM dim_cara_bayar", connection)
C:\Users\hanif\AppData\Local\Temp\ipykernel_14112\696343404.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dim_cara_keluar = pd.read_sql("SELECT * FROM dim_c

In [3]:
import pandas as pd

# Langkah 1: Read Data dari CSV
df = pd.read_csv('dataetl.csv')

# Langkah 2: Select Kolom Tanggal
# Pastikan kolom 'tanggal' ada di data Anda
tanggal_df = df[['tanggal']]

# Langkah 3: Transformasi Tanggal ke Day, Month, Year
# Ubah kolom 'tanggal' menjadi tipe datetime
tanggal_df['tanggal'] = pd.to_datetime(tanggal_df['tanggal'], errors='coerce')

# Ekstrak hari, bulan, dan tahun
tanggal_df['day'] = tanggal_df['tanggal'].dt.day
tanggal_df['month'] = tanggal_df['tanggal'].dt.month
tanggal_df['year'] = tanggal_df['tanggal'].dt.year

# Langkah 4: Drop Duplicate
# Hapus baris duplikat
tanggal_df = tanggal_df.drop_duplicates()

# Langkah 5: Tambahkan Kolom ID
# Tambahkan kolom 'ID' yang berisi nilai increment dari 1
tanggal_df['ID'] = range(1, len(tanggal_df) + 1)

# Tampilkan hasil akhir
print(tanggal_df)

try:
    # Membuat koneksi ke server MySQL pada XAMPP
    connection = mysql.connector.connect(host='localhost',
                                         database='db_mnatsir', 
                                         user='root',     
                                         password='')

    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("USE db_mnatsir")  # Ganti nama_database
        
        # Memuat data ke dalam tabel pada server MySQL
        for index, row in df_no_duplicates.iterrows():
            cursor.execute("INSERT INTO dim_waktu (id_waktu, day, month, year) VALUES (%s, %s, %s, %s)", (row['id_waktu'], row['day'], row['month'], row['year']))
            connection.commit()
        
        print("Data berhasil dimuat ke MySQL")
            
except mysql.connector.Error as e:
    print("Error:", e)
    
finally:
    if 'connection' in locals():
        connection.close()
        print("Koneksi ke MySQL ditutup")


         tanggal  day  month  year    ID
0     2020-01-01    1      1  2020     1
5     2020-01-02    2      1  2020     2
15    2020-01-03    3      1  2020     3
23    2020-01-04    4      1  2020     4
34    2020-01-05    5      1  2020     5
...          ...  ...    ...   ...   ...
12712 2020-11-26   26     11  2020  1366
12750 2021-01-17   17      1  2021  1367
13049 2021-09-23   23      9  2021  1368
20011 2021-08-19   19      8  2021  1369
30819 2023-10-06    6     10  2023  1370

[1370 rows x 5 columns]


C:\Users\hanif\AppData\Local\Temp\ipykernel_11488\1574155215.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tanggal_df['tanggal'] = pd.to_datetime(tanggal_df['tanggal'], errors='coerce')
C:\Users\hanif\AppData\Local\Temp\ipykernel_11488\1574155215.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tanggal_df['day'] = tanggal_df['tanggal'].dt.day
C:\Users\hanif\AppData\Local\Temp\ipykernel_11488\1574155215.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra